In [163]:
import plotly.offline as pyo
import plotly.graph_objects as go
import ipywidgets as widgets
import numpy as np
import pandas as pd
from datetime import datetime

In [164]:
#pyo.init_notebook_mode(connected=True)

In [165]:
from os import getcwd, path
data_path = path.join(path.split(getcwd())[0], '00_data')

In [166]:
df = pd.read_csv(path.join(data_path, 'nlp_social.csv'), sep = ",")

In [167]:
is_ta = df['Source'] == 'TripAdvisor'
is_tw = df['Source'] == 'Twitter'
df_ta = df[is_ta]
df_tw = df[is_tw]

In [168]:
df_tw['Date'] = pd.to_datetime(df_tw['Date'], format='%Y-%m-%d %H:%M:%S')
df_tw['day'] = df_tw['Date'].map(lambda x: x.strftime('%Y-%m-%d'))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [170]:
# Select only tweets which relates to each topic
df_seat = df_tw[df_tw['Seat']!='[]']
df_staff = df_tw[df_tw['Staff Service']!='[]']
df_fb = df_tw[df_tw['Food & Beverages']!='[]']
df_ent = df_tw[df_tw['Inflight Entertainment']!='[]']
df_gserv = df_tw[df_tw['Ground Service']!='[]']

# Group them by date and average them
df_seat = df_seat.groupby('day').Seat_Polarity.mean().reset_index(name='seat')
df_staff = df_staff.groupby('day').Staff_Service_Polarity.mean().reset_index(name='staff_service')
df_fb = df_fb.groupby('day').FoodBeverages_Polarity.mean().reset_index(name='foodbeverage')
df_ent = df_ent.groupby('day').InflightEntertainment_Polarity.mean().reset_index(name='inflightent')
df_gserv = df_gserv.groupby('day').GroundService_Polarity.mean().reset_index(name='gserv')

In [174]:
df_tw.rename(columns={
    "Seat": "Seat", 
    "Staff Service": "Staff_Service", 
    "Food & Beverages": "FoodBeverages",
    "Inflight Entertainment": "InflightEntertainment", 
    "Ground Service": "GroundService"}, inplace = True)

In [186]:
def select_best_tweet_avg(subject, day, selected_nb = 3, df = df_tw):
    subject_polarity = '{}_Polarity'.format(subject).replace('&', '').replace(' ','_')
    
    df_subject_day =  df_tw[[subject, subject_polarity, 'day']] \
        [df_tw[subject]!='[]'] \
        [df_tw['day'] == day]
    
    mean_subject_day = df_subject_day[subject_polarity].mean()
    df_subject_day['mean_subject_day'] = abs(df_subject_day[subject_polarity] - mean_subject_day)
    df_subject_day.set_index('mean_subject_day', inplace = True)
    df_subject_day.sort_index(ascending = True, inplace = True)
    
    return pd.DataFrame(data = [[day, subject, df_subject_day.iloc[0:3][subject].values]],
                            columns = ['day', 'subject', 'avg_tweet'])
    

In [187]:
subjects_list = [
    'Seat',
    'Staff_Service',
    'FoodBeverages',
    'InflightEntertainment',
    'GroundService'
]
days_list = ['2020-01-{}'.format(day) for day in range(17, 27)]

avg_tweet_day_subject = []
for subject in subjects_list:
    for day in days_list:
        avg_tweet_day_subject.append(select_best_tweet_avg(subject, day))
avg_tweet_day_subject = pd.concat(avg_tweet_day_subject, ignore_index = True)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [200]:
avg_tweet_day_subject

,day,subject,avg_tweet
0,2020-01-17,Seat,[['See that my seats have been changed even th...
1,2020-01-18,Seat,[[' need to work harder - paid for first class...
2,2020-01-19,Seat,[[' can you explain why my elderly mother in l...
3,2020-01-20,Seat,[[': Malaysia has sent back 150 containers of ...
4,2020-01-21,Seat,[['i literally almost cried cause i got rebook...
5,2020-01-22,Seat,[['And this....how far do USA and Canada back ...
6,2020-01-23,Seat,[['Lufthansa flight from to is heading back...
7,2020-01-24,Seat,"[[""Legally you've only got 4 days to get me co..."
8,2020-01-25,Seat,[['Need one of those vintage logos taken back ...
9,2020-01-26,Seat,"[[': We are signing a centre back', 'A full on..."


In [10]:
# Create a new dataframe with the day and the average sentiment for each topic
df_new = df_seat.merge(df_staff, left_on = 'day', right_on = 'day')
df_new = df_new.merge(df_fb, left_on = 'day', right_on = 'day')
df_new = df_new.merge(df_ent, left_on = 'day', right_on = 'day')
df_new = df_new.merge(df_gserv, left_on = 'day', right_on = 'day')
df_new

,day,seat,staff_service,foodbeverage,inflightent,gserv
0,2020-01-16,-0.007487,0.014834,-0.166667,0.179293,0.019531
1,2020-01-17,-0.008384,-0.005114,0.030808,0.115341,-0.002608
2,2020-01-18,0.066038,0.098374,0.265278,0.070288,-0.046610
3,2020-01-19,0.082481,-0.039177,-0.000084,0.120954,0.017845
4,2020-01-20,0.062466,-0.066533,0.005889,0.037413,-0.029527
5,2020-01-21,0.040938,0.050725,0.068089,0.218182,0.020452
6,2020-01-22,0.031507,0.042885,-0.040122,0.215924,0.079406
7,2020-01-23,0.027704,0.080108,0.097826,0.143245,0.081652
8,2020-01-24,0.046974,-0.097874,0.091270,0.050000,0.048882
9,2020-01-25,0.077273,-0.080931,-0.004066,0.090362,0.053673


In [16]:
x_data = np.array([df_new['day']]*5)

y_data = np.array([df_new['seat'],
                   df_new['staff_service'],
                   df_new['foodbeverage'],
                   df_new['inflightent'],
                   df_new['gserv']])

In [17]:
x_data

array([['2020-01-16', '2020-01-17', '2020-01-18', '2020-01-19',
        '2020-01-20', '2020-01-21', '2020-01-22', '2020-01-23',
        '2020-01-24', '2020-01-25', '2020-01-26'],
       ['2020-01-16', '2020-01-17', '2020-01-18', '2020-01-19',
        '2020-01-20', '2020-01-21', '2020-01-22', '2020-01-23',
        '2020-01-24', '2020-01-25', '2020-01-26'],
       ['2020-01-16', '2020-01-17', '2020-01-18', '2020-01-19',
        '2020-01-20', '2020-01-21', '2020-01-22', '2020-01-23',
        '2020-01-24', '2020-01-25', '2020-01-26'],
       ['2020-01-16', '2020-01-17', '2020-01-18', '2020-01-19',
        '2020-01-20', '2020-01-21', '2020-01-22', '2020-01-23',
        '2020-01-24', '2020-01-25', '2020-01-26'],
       ['2020-01-16', '2020-01-17', '2020-01-18', '2020-01-19',
        '2020-01-20', '2020-01-21', '2020-01-22', '2020-01-23',
        '2020-01-24', '2020-01-25', '2020-01-26']], dtype=object)

In [18]:
y_data

array([[-7.48737374e-03, -8.38428932e-03,  6.60380592e-02,
         8.24810606e-02,  6.24664726e-02,  4.09382284e-02,
         3.15071166e-02,  2.77036544e-02,  4.69737253e-02,
         7.72727273e-02,  8.57883083e-02],
       [ 1.48343904e-02, -5.11363636e-03,  9.83737374e-02,
        -3.91768293e-02, -6.65327937e-02,  5.07245532e-02,
         4.28851010e-02,  8.01084506e-02, -9.78741361e-02,
        -8.09307359e-02,  2.32332643e-02],
       [-1.66666667e-01,  3.08080808e-02,  2.65277778e-01,
        -8.40336134e-05,  5.88852232e-03,  6.80894309e-02,
        -4.01220469e-02,  9.78259676e-02,  9.12698413e-02,
        -4.06617192e-03, -2.37068966e-03],
       [ 1.79292929e-01,  1.15340909e-01,  7.02876984e-02,
         1.20954106e-01,  3.74125874e-02,  2.18181818e-01,
         2.15923945e-01,  1.43245243e-01,  5.00000000e-02,
         9.03622693e-02,  1.56939394e-01],
       [ 1.95308347e-02, -2.60804769e-03, -4.66095238e-02,
         1.78447784e-02, -2.95265152e-02,  2.04524174e-02,
  

In [31]:
import plotly.offline as pyo
import plotly.graph_objects as go

title = 'Main Source for News'
labels = ['Seat', 'Staff Service', 'Food & Beverage', 'Inflight Entertainment','Ground Service']
colors = ['rgb(67,67,67)', 'rgb(115,115,115)', 'rgb(49,130,189)', 'rgb(189,189,189)','rgb(40,140,140)']

mode_size = [10, 10, 10, 10, 10]
line_size = [4, 4, 4, 4, 4]

x_data = np.array([df_new['day']]*5)

y_data = np.array([df_new['seat'],
                   df_new['staff_service'],
                   df_new['foodbeverage'],
                   df_new['inflightent'],
                   df_new['gserv']])

fig = go.Figure()

for i in range(0,5):
    fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode='lines',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
    ))

    # endpoints
    fig.add_trace(go.Scatter(
        x=[x_data[i][0], x_data[i][-1]],
        y=[y_data[i][0], y_data[i][-1]],
        mode='markers',
        marker=dict(color=colors[i], size=mode_size[i])
    ))

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=False,
    plot_bgcolor='white'
)

annotations = []

# Adding labels
for y_trace, label, color in zip(y_data, labels, colors):
    # labeling the left_side of the plot
    annotations.append(dict(xref='paper', 
                            x=0.05, 
                            y=y_trace[0],
                            xanchor='right', 
                            yanchor='middle',
                            text=label + ' {}‰'.format(round(y_trace[0]*10, 2)),
                            font=dict(family='Arial',
                                      size=16),
                            showarrow=False))
    # labeling the right_side of the plot
    annotations.append(dict(xref='paper', 
                            x=0.95, 
                            y=y_trace[10],
                            xanchor='left', 
                            yanchor='middle',
                            text='{}‰'.format(round(y_trace[10]*10, 2)),
                            font=dict(family='Arial',
                                      size=16),
                                      showarrow=False))
# Title
annotations.append(dict(xref='paper', 
                        yref='paper', 
                        x=0.0, 
                        y=1.05,
                        xanchor='left', 
                        yanchor='bottom',
                        text='Social Media Sentiment over the last 7 days',
                        font=dict(family='Arial',
                                  size=30,
                                  color='rgb(37,37,37)'),
                                  showarrow=False))
# Source
annotations.append(dict(xref='paper', 
                        yref='paper', 
                        x=0.5, 
                        y=-0.1,
                        xanchor='center', 
                        yanchor='top',
                        text='Source: Twitter',
                        font=dict(family='Arial',
                                  size=12,
                                  color='rgb(150,150,150)'),
                                  showarrow=False))

fig.update_layout(annotations=annotations)

fig.show()
#pyo.iplot(fig)

0.019530834662413607